In [1]:
import string

def eliminar_puntuacion(texto):
    # Define una cadena de caracteres de puntuación
    puntuacion = string.punctuation
    
    # Inicializa una cadena vacía para almacenar el texto sin puntuación
    texto_sin_puntuacion = ""
    
    # Itera a través de cada carácter en el texto
    for caracter in texto:
        # Si el carácter no está en la cadena de puntuación, agrégalo al texto sin puntuación
        if caracter not in puntuacion:
            texto_sin_puntuacion += caracter
    
    return texto_sin_puntuacion.strip()

In [2]:
import torch

In [3]:
############# code changes ###############
import intel_extension_for_pytorch as ipex
############# code changes ###############

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from datasets import DatasetDict, Dataset
import pandas as pd
df_train = pd.read_pickle("data/train_df_new_wa.pkl")
df_test = pd.read_pickle("data/validation_df_new_wa.pkl")

In [6]:
df_train

,question,human_ans_indices,review,human_ans_spans,Answer
0,What was the result of Stan Wawrinka's match a...,"(1207, 1222)","Lille, France (CNN) -- Roger Federer and Stan ...",6-1 3-6 6-3 6-2,6-1 3-6 6-3 6-2
1,What is the nickname given to Fauja Singh by h...,"(420, 436)",Hong Kong (CNN) -- Not your average great-grea...,turbaned tornado,turbaned tornado
2,Who was kidnapped?,"(173, 185)",(CNN) -- Four months after a criminal investig...,Kyron Horman,Kyron Horman
3,Who are people upset with?,"(36, 51)",(CNN) -- Los Angeles Clippers owner Donald Ste...,Donald Sterling,Donald Sterling
4,How did she die?,"(32, 64)",(CNN) -- The police officer who fatally shot a...,fatally shot a 93-year-old woman,she was fatally shot
...,...,...,...,...,...
59057,Who was one part for?,"(518, 538)",CHAPTER V. \n\n_THE WAR BETWEEN CÆSAR AND POMP...,"one for the lawyers,",The lawyers.
59058,did their father answer the door?,"(468, 532)",CHAPTER XX. \n\nFOLLOWING ALLEN. \n\nHal was a...,"The door was closed, and then Andy McCabe answ...",yes
59059,"What is the political, cultural and economic c...","(0, 10)","Bratislava ( or ; , or """" ) is the capital of ...",bratislava,bratislava
59060,How?,"(1117, 1161)",(CNN) -- Obsession often brings joy and sorrow...,and then get some for my 31-year-old husband,she buys them for him in shops


In [7]:
df_train['is_question'] = df_train['Answer'].apply(lambda x: 1 if eliminar_puntuacion(x).lower() == 'yes' else (0 if eliminar_puntuacion(x).lower() == 'no' or eliminar_puntuacion(x).lower() == 'not' else None))
df_train = df_train[['question', 'is_question', 'human_ans_spans']].rename(columns={'human_ans_spans': 'text', 'is_question': 'label'})
df_train['question'] = df_train['question'] + '\n'
df_train['text'] = df_train['text'] + '\n'
df_train['answer0'] = 'no'
df_train['answer1'] = 'yes'

df_test['is_question'] = df_test['Answer'].apply(lambda x: 1 if eliminar_puntuacion(x).lower() == 'yes' else (0 if eliminar_puntuacion(x).lower() == 'no' or eliminar_puntuacion(x).lower() == 'not' else None))
df_test = df_test[['question', 'is_question', 'human_ans_spans']].rename(columns={'human_ans_spans': 'text', 'is_question': 'label'})
df_test['question'] = df_test['question'] + '\n'
df_test['text'] = df_test['text'] + '\n'
df_test['answer0'] = 'no'
df_test['answer1'] = 'yes'

In [8]:
df_train.dropna(subset=['label'], inplace = True)
df_train = df_train[['question', 'label', 'text', 'answer0', 'answer1']].reset_index()
df_train

,index,question,label,text,answer0,answer1
0,7,did he cry a lot?\n,0.0,Baby Will never cried\n,no,yes
1,14,Did the riders leave late?\n,0.0,were on their horses' backs at an early hour. \n,no,yes
2,18,Did Walter lie about what happened?\n,0.0,Then Walter gave a truthful account \n,no,yes
3,19,do they use it in astronomy?\n,1.0,"Myr is deprecated in geology, but in astronomy...",no,yes
4,21,Is it the largest city in Armenia?\n,1.0,largest city of Armenia \n,no,yes
...,...,...,...,...,...,...
10088,59044,Can singles get those distinctions as well?\n,1.0,albums and singles\n,no,yes
10089,59046,Did they ever run out of money?\n,1.0,filed for Chapter 11 bankruptcy\n,no,yes
10090,59051,Was he present when police showed up?\n,0.0,only one of the parties involved was there\n,no,yes
10091,59058,did their father answer the door?\n,1.0,"The door was closed, and then Andy McCabe answ...",no,yes


In [9]:
df_test.dropna(subset=['label'], inplace = True)
df_test = df_test[['question', 'label', 'text', 'answer0', 'answer1']].reset_index()
df_test

,index,question,label,text,answer0,answer1
0,2,Did he ever represent his country in Winter Ol...,0.0,Keung has never represented Switzerland at a W...,no,yes
1,3,Is he a doctor?\n,1.0,"Dr. Kerns H. Powers, \n",no,yes
2,10,Does he have any conditions?\n,1.0,"Bales has not entered a plea in the case, thou...",no,yes
3,15,Do they know when to expect Joe?\n,0.0,no\n,no,yes
4,29,Does she live in a house?\n,0.0,They lived in an apartment.\n,no,yes
...,...,...,...,...,...,...
1109,6552,Was anyone in it?\n,0.0,the vacant building\n,no,yes
1110,6553,Is he ahead in the race?\n,0.0,"but the race is closer in Michigan, with the p...",no,yes
1111,6557,were there 1000 people killed ?\n,0.0,killing at least 14\n,no,yes
1112,6558,did she she apologize?\n,1.0,"Sarah said in a quiet voice, ""I'm sorry, Mom.\n",no,yes


In [10]:
dataset_dict = DatasetDict({
    'train': Dataset.from_pandas(df_train),
    'test': Dataset.from_pandas(df_test)
})
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['index', 'question', 'label', 'text', 'answer0', 'answer1'],
        num_rows: 10093
    })
    test: Dataset({
        features: ['index', 'question', 'label', 'text', 'answer0', 'answer1'],
        num_rows: 1114
    })
})

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [12]:
ending_names = ["answer0", "answer1"]
n = len(ending_names)

def preprocess_function(examples):
    first_sentences = [[context] * n for context in examples["text"]]
    question_headers = examples["question"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + n] for i in range(0, len(v), n)] for k, v in tokenized_examples.items()}

In [13]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/10093 [00:00<?, ? examples/s]

Map:   0%|          | 0/1114 [00:00<?, ? examples/s]

In [14]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [15]:
import evaluate

accuracy = evaluate.load("accuracy")

In [16]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

**Train**

In [17]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir="yn_answer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    use_ipex=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:484: UserWarning: Split Master Weight feature is not supported on XPU for now, disabled.
  warnings.warn("Split Master Weight feature is not supported on XPU for now, disabled.")
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:500: UserWarning: Weight Prepack and Sample Input are both disabled on XPU. The Onednn Layout is automatically applied.
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.485800,0.340122,0.855476
2,0.266700,0.462415,0.844704
3,0.142400,0.769005,0.851885


/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:494: UserWarning: To reduce device memory usage on XPU, optimization are done inplace, setting the inplace argument to True.
  warnings.warn(
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:527: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn("Linear BatchNorm folding failed during the optimize process.")
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:484: UserWarning: Split Master Weight feature is not supported on XPU for now, disabled.
  warnings.warn("Split Master Weight feature is not supported on XPU for now, disabled.")
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:494: UserWarning: To reduce device memory usage on XPU, optimization are done in

TrainOutput(global_step=1893, training_loss=0.24637082337199603, metrics={'train_runtime': 3871.8313, 'train_samples_per_second': 7.82, 'train_steps_per_second': 0.489, 'total_flos': 3133643033364696.0, 'train_loss': 0.24637082337199603, 'epoch': 3.0})

In [19]:
trainer.push_to_hub()

'https://huggingface.co/roaltopo/yn_answer/tree/main/'